### Obvious things to tweak


Number of layers
Nodes per layer
Dense layer at the end or not

In [2]:
import time
dense_layers = [1, 2]
layer_sizes = [64, 128]
conv_layers = [4]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

4-conv-64-nodes1-dense-1540675743
4-conv-128-nodes1-dense-1540675743
4-conv-64-nodes2-dense-1540675743
4-conv-128-nodes2-dense-1540675743


In [18]:
import tensorflow as tf
import keras
from keras import backend as K

# NOTE: To shrink the VRAM fraction add this to ConfigProto
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# K.tf.ConfigProto(gpu_options=gpu_options)

cfg = K.tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1})
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import normalize
from keras.callbacks import TensorBoard

import pickle
import time

datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.15,
                             zoom_range=0.1, channel_shift_range=10., horizontal_flip=True)

X = pickle.load(open("data/pickled/X.pickle", "rb"))
y = pickle.load(open("data/pickled/y.pickle", "rb"))

X = keras.utils.normalize(X, axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2)

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            K.clear_session()
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs_r2/{}'.format(NAME))
            
            model = Sequential()

            # the 128 can be anything
            # 3,3 is the window of the convolution (3 pixels X 3 pixels)
            # X.shape[1:] is 128, 128, 1 based ont he fact that I transformed the images to 128X128 pixes
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
            model.add(Dropout(0.2))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3,3)))
                model.add(Activation("relu"))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))

            # Output Layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
            
            model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                               steps_per_epoch = len(X) / 32,
                               epochs = 10,
                               validation_data=(X_val, y_val),
                               callbacks=[tensorboard])

            model.fit(X, y, batch_size=32, epochs=10, validation_split=0.1, callbacks=[tensorboard])

Train on 22451 samples, validate on 2495 samples
Epoch 1/10
22451/22451 [==============================] - 17s 759us/step - loss: 0.6488 - acc: 0.6194 - val_loss: 0.6195 - val_acc: 0.6425
Epoch 2/10
22451/22451 [==============================] - 17s 745us/step - loss: 0.5812 - acc: 0.7025 - val_loss: 0.5884 - val_acc: 0.6910
Epoch 3/10
22451/22451 [==============================] - 17s 751us/step - loss: 0.5173 - acc: 0.7517 - val_loss: 0.4794 - val_acc: 0.7792
Epoch 4/10
22451/22451 [==============================] - 17s 751us/step - loss: 0.4756 - acc: 0.7782 - val_loss: 0.4830 - val_acc: 0.7671
Epoch 5/10
22451/22451 [==============================] - 17s 759us/step - loss: 0.4517 - acc: 0.7936 - val_loss: 0.4388 - val_acc: 0.7980
Epoch 6/10
22451/22451 [==============================] - 17s 749us/step - loss: 0.4280 - acc: 0.8061 - val_loss: 0.4574 - val_acc: 0.7820
Epoch 7/10
22451/22451 [==============================] - 17s 752us/step - loss: 0.4084 - acc: 0.8127 - val_loss: 0.4

In [4]:
# Round two attemps
import tensorflow as tf
import keras
from keras import backend as K

# NOTE: To shrink the VRAM fraction add this to ConfigProto
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
# K.tf.ConfigProto(gpu_options=gpu_options)

cfg = K.tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1})
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import normalize
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator

import pickle
import time

datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.15,
                             zoom_range=0.1, channel_shift_range=10., horizontal_flip=True)

X = pickle.load(open("data/pickled/X.pickle", "rb"))
y = pickle.load(open("data/pickled/y.pickle", "rb"))

# X = keras.utils.normalize(X, axis=1)

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False)

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            K.clear_session()
            NAME = "{}-conv-{}-nodes{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            tensorboard = TensorBoard(log_dir='logs_2/{}'.format(NAME))
            
            model = Sequential()

            # the 128 can be anything
            # 3,3 is the window of the convolution (3 pixels X 3 pixels)
            # X.shape[1:] is 128, 128, 1 based ont he fact that I transformed the images to 128X128 pixes
            model.add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model.add(Activation("relu"))
#             model.add(Dropout(0.2))
            model.add(MaxPooling2D(pool_size=(2,2)))
            
            for l in range(conv_layer-1):
                model.add(Conv2D(2*layer_size, (3,3)))
                model.add(Activation("relu"))
#                 model.add(Dropout(0.5))
                model.add(MaxPooling2D(pool_size=(2,2)))
            
            model.add(Flatten())
            for l in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation("relu"))
#                 model.add(Dropout(0.5))

            # Output Layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
            
#             model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val), callbacks=[tensorboard])
            model.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                               steps_per_epoch = len(X) / 32,
                               epochs = 12,
                               validation_data=(X_val, y_val),
                               callbacks=[tensorboard])

Epoch 1/12
780/779 [==============================] - 38s 48ms/step - loss: 0.6663 - acc: 0.5857 - val_loss: 5.6177 - val_acc: 0.6413
Epoch 2/12
780/779 [==============================] - 37s 47ms/step - loss: 0.5481 - acc: 0.7220 - val_loss: 4.0083 - val_acc: 0.7385
Epoch 3/12
780/779 [==============================] - 37s 47ms/step - loss: 0.4580 - acc: 0.7825 - val_loss: 3.7140 - val_acc: 0.7595
Epoch 4/12
780/779 [==============================] - 37s 48ms/step - loss: 0.3981 - acc: 0.8181 - val_loss: 3.2478 - val_acc: 0.7878
Epoch 5/12
780/779 [==============================] - 37s 47ms/step - loss: 0.3580 - acc: 0.8388 - val_loss: 3.0616 - val_acc: 0.7994
Epoch 6/12
780/779 [==============================] - 37s 47ms/step - loss: 0.3303 - acc: 0.8550 - val_loss: 3.6416 - val_acc: 0.7613
Epoch 7/12
780/779 [==============================] - 37s 47ms/step - loss: 0.3077 - acc: 0.8660 - val_loss: 2.8708 - val_acc: 0.8152
Epoch 8/12
780/779 [==============================] - 37s 47ms